# Introduction to Variational Autoencoders with Pyro
Welcome to this introductory Jupyter Notebook on Variational Autoencoders (VAEs) built using Pyro! In this tutorial, we will explore the fascinating world of generative models and dive into the inner workings of VAEs.

Generative models are a powerful class of machine learning algorithms that can learn to generate new data samples similar to the training data. VAEs, in particular, are a type of generative model that combines techniques from both deep learning and probabilistic modeling. They allow us to capture complex patterns and generate new samples from high-dimensional data distributions.

# Sections
0. Setup
1. Defining the Encoder
2. Defining the Decoder
3. Defining the VAE (Model & Guide)
4. Inference and Results

## 0. Setup
In this section, we will set up the necessary environment and dependencies for working with Variational Autoencoders (VAEs) in Pyro. We will install any required packages, import the necessary libraries, and load the dataset (if applicable). It is important to have a properly configured environment before proceeding to the subsequent sections.

In [ ]:
!pip3 install -r requirements.txt

import pyro
import torch
from pyro import distributions as dist
from pyro.nn import PyroModule, PyroSample, PyroParam
from pyro.contrib.examples.util import MNIST
import pyro.poutine as poutine
import torchvision.transforms as transforms
from torch import nn
import numpy as np
from tqdm import tqdm
from PIL import Image
from sklearn.manifold import TSNE
import plotly.express as px
from torchsummary import summary
from itertools import chain
import plotly.express as px

In [ ]:
# Set up the datatloaders for training and testing data
def setup_data_loaders(batch_size=128):
    root = './data'
    download = True
    trans = transforms.ToTensor()
    train_set = MNIST(
        root=root,
        train=True,
        transform=trans,
        download=download
    )
    test_set = MNIST(
        root=root,
        train=False,
        transform=trans,
        download=download
    )

    train_loader = torch.utils.data.DataLoader(
        dataset=train_set,
        batch_size=batch_size,
        shuffle=True,
    )

    test_loader = torch.utils.data.DataLoader(
        dataset=test_set,
        batch_size=batch_size,
        shuffle=False,
    )

    return train_loader, test_loader

## 1. Defining the Encoder
The encoder is a crucial component of the Variational Autoencoder (VAE) architecture. In this section, we will define the encoder network, which takes an input data sample and maps it to the corresponding latent space representation. We will explore various architectures and techniques for constructing the encoder network, such as fully connected layers, convolutional layers, and activation functions. Understanding the encoder's role and designing an effective architecture is essential for the overall performance of the VAE.

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Set up layers
        self.conv1 = None
        self.conv2 = None
        self.batchnorm = None
        self.fc_mean = None
        self.fc_std = None
        
        # Set up non-linearities
        pass
        
    def forward(self, X):
        z_loc = None
        z_scale = None
        
        return z_loc, z_scale

In [ ]:
summary(Encoder(), (1, 28, 28))

## 2. Defining the Decoder
The decoder complements the encoder in the VAE framework. It takes a latent space representation and reconstructs the original input data sample. In this section, we will define the decoder network, which is responsible for generating the output based on the latent variables. We will discuss different architectural choices for the decoder, including deconvolutional layers, transposed convolutional layers, and non-linear activation functions. A well-designed decoder is crucial for generating high-quality samples from the learned latent space.

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Set up layers
        self.fc_mean_std = None
        self.deconv2 = None
        self.deconv1 = None
        self.fc_out = None
        
        # Set up non-linearities and transformations
        pass

        
    def forward(self, z):
        loc_out = None
        
        return loc_out

In [ ]:
summary(Decoder(), (32,))

## 3. Defining the VAE (Model & Guide)
In this section, we will bring together the encoder and decoder components to define the complete Variational Autoencoder (VAE) model using Pyro. We will specify the probabilistic model and the guide, which will be used for posterior inference. We will define the priors, likelihood functions, and latent variables, as well as discuss the necessary modifications to the standard VAE formulation. Understanding the model and guide definitions is essential for training and inference in VAEs.

In [ ]:
class VAE(nn.Module):
    def __init__(self, in_channels: int = 1, hidden_channels: int = 32, latent_channels: int = 1):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

        self.z_dim = 32
        
    def model(self, X=None, samples_to_generate=None):
        pyro.module("decoder", self.decoder)
        with None:
            z_loc = None
            z_scale = None
            z = None

            loc_out = None
            pyro.sample(None)
            
            return loc_out
            
    def guide(self, X, samples_to_generate=None):
        pyro.module("encoder", self.encoder)

        with None:
            z_loc, z_scale = None, None
            pyro.sample(None)
            
    def reconstruct_img(self, x):
        # encode image x
        z_loc, z_scale = self.encoder(x)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        loc_img = self.decoder(z)
        return loc_img

## 4. Training the VAE

In this section, we will dive into the training process of our Variational Autoencoder (VAE). Now that we have defined the VAE model and guide, it's time to optimize its parameters and make the model learn from our data. Training a VAE involves maximizing the evidence lower bound (ELBO) objective, which balances the reconstruction loss and the regularization term imposed by the variational distribution.

In [ ]:
vae = None
optimizer = None
svi = pyro.infer.SVI(
    model=None,
    guide=None,
    optim=None,
    loss=None
)

In [ ]:
EPOCHS = None

train_loader, test_loader = setup_data_loaders()

loss = []

pyro.clear_param_store()

for epoch in tqdm(range(EPOCHS), total=EPOCHS):
    epoch_loss = 0
    for batch, _ in train_loader:
        pass
        
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    loss.append(total_epoch_loss_train)
    if (epoch % 10) == 0:
        print(total_epoch_loss_train)

px.line(loss)

## 5. Generating Images with the Trained VAE

In this section, we will explore the capabilities of our trained Variational Autoencoder (VAE) by generating new images. By sampling from the learned latent space, we can generate novel data samples that capture the underlying patterns and variations present in the training dataset. We will examine the test loss, visualize the test images in a T-SNE projection of the latent space, and finally generate new images by sampling from the latent space distribution.

### 5.1 Test Loss

In this sub-section, we evaluate the performance of our trained VAE by calculating the test loss across all the testing samples. The test loss provides an indication of how well our VAE generalizes to unseen data. By comparing the test loss to the training loss, we can assess if the model exhibits overfitting or underfitting tendencies. A lower test loss suggests that the VAE has learned meaningful representations and can reconstruct the testing images effectively.

In [ ]:
def evaluate(svi, test_loader):
    # initialize loss accumulator
    test_loss = 0.
    # compute the loss over the entire test set
    for x, _ in test_loader:
        # compute ELBO estimate and accumulate loss
        test_loss += svi.evaluate_loss(x)
    normalizer_test = len(test_loader.dataset)
    total_epoch_loss_test = test_loss / normalizer_test
    return total_epoch_loss_test

In [ ]:
evaluate(svi=svi, test_loader=test_loader)

### 5.2 Test Images in T-SNE

In this sub-section, we visualize the test images in a T-SNE projection of the latent space. T-SNE (t-Distributed Stochastic Neighbor Embedding) is a dimensionality reduction technique commonly used for visualizing high-dimensional data in a lower-dimensional space. By projecting the latent representations of the test images onto a 2D or 3D space, we can gain insights into the clustering and structure of the latent space. This visualization allows us to understand how well the VAE has captured the underlying data distribution and if similar images are grouped together.

In [ ]:
# Put test images in latent

latent_images = [
    vae.encoder(img)[0].detach().cpu().numpy() for img, _ in test_loader
]
labels = [label.detach().cpu().numpy().astype(str) for _, label in test_loader]
labels = list(chain.from_iterable(labels))
latent_images = np.concatenate(latent_images, axis=0)

tsne = TSNE(n_components=2, n_jobs=-1, random_state=0, metric="cosine", verbose=10)
latent_embedding = tsne.fit_transform(latent_images)
px.scatter(x=latent_embedding[:, 0], y=latent_embedding[:, 1], color=labels)

### 5.3 New Images

In this sub-section, we harness the power of the trained VAE to generate new images. By sampling from the learned latent space distribution, we can explore the generative capabilities of the model. We will randomly sample latent vectors from the prior distribution or systematically traverse the latent space to observe the variations in the generated images. This provides an exciting opportunity to create novel, never-before-seen images that resemble the patterns learned during training. We can adjust the latent variables to influence the generated images and explore the continuous variations in the generated samples.

By the end of this section, you will have a comprehensive understanding of the VAE's performance on the test dataset, visual insights into the latent space using T-SNE, and the ability to generate new images. Let's proceed and unlock the generative power of our trained VAE!

In [ ]:
noise = torch.zeros([28, 28])
sample_loc = vae.model(noise).reshape(-1, 1, 28, 28)
img = vae.reconstruct_img(sample_loc).detach().cpu().numpy().reshape(-1, 28, 28)

def gen(imgs):
    for img in imgs:
        yield img
        
img = gen(imgs=img)

In [ ]:
Image.fromarray(np.uint8(next(img) * 255)).resize((100, 100))